In [1]:
import tensorflow as tf
import numpy as np
import os

## Cargar el Corpus

In [2]:
archivo = 'lyrics_data_esp.txt' # lyrics

# guardar el archivo (corpus) en una variable
texto = open(archivo, 'rb').read()
texto = texto.decode(encoding='utf-8')

# imprimir numero de caracteres del corpus
# y los primeros 100 caracteres
print('Total de caracteres en el corpus: ', len(texto))
print('Primeros 100 caracteres del corpus:\n', texto[:100])

Total de caracteres en el corpus:  2333941
Primeros 100 caracteres del corpus:
 en la improvisación
lamentablemente yo muerdo como león
yo te prendo fuego, el concepto es juego
y c


## Vectorizar el texto  

• Dar un número de índice a cada carácter único.  
• Ejecutar un ciclo for en el corpus e indexar cada carácter en todo el texto.

In [3]:
# extraer los caracteres unicos en el corpus
vocab = sorted(set(texto))
print('Caracteres unicos en el corpus: ', len(vocab))
print('Parte del set de caracteres:\n', vocab[:10])

# dar a cada caracter un numero de indice
char2idx = {u:i for i, u in enumerate(vocab)}

# copiar los elementos del conjunto único a una matriz NumPy para su uso posterior
# en la decodificación de las predicciones
idx2char = np.array(vocab)

# vectorizar el texto con un bucle for simple donde revisamos cada carácter del
# texto y asignamos su valor de índice correspondiente y guardamos todos los
# valores de índice como una nueva lista
texto_a_int = np.array([char2idx[c] for c in texto])

# print('\n',char2idx)
# print('\n',idx2char)
# print('\n',texto_a_int)

Caracteres unicos en el corpus:  73
Parte del set de caracteres:
 ['\n', ' ', '!', '"', '#', '&', "'", '(', ')', ',']


## Crear el Dataset  

El metodo from_tensor_slices del modulo Dataset crea un objeto TensorFlow Dataset a partir de nuestro objeto texto_a_int, y los dividiremos en lotes. La longitud de cada entrada del conjunto de datos esta limitada a 100 caracteres

In [4]:
char_dataset = tf.data.Dataset.from_tensor_slices(texto_a_int)
seq_tam = 100   # El maximo para una entrada unica
secuencias = char_dataset.batch(seq_tam+1, drop_remainder=True)

print(secuencias)

<BatchDataset shapes: (101,), types: tf.int64>


El objeto 'secuencias' contiene secuencias de caracteres, pero crearemos una tupla de estas secuencias simplemente para alimentar el modelo RNN conla funcion map

In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = secuencias.map(split_input_target)
print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


Mezclar el conjunto de datos y lo dividirlo en lotes de 64 oraciones (sentencias)

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


## Construir el modelo  

Construir el modelo de manera que acepte 64 oraciones de entrada a la vez. Despues de entenar el modelo, ingresaremos oraciones individuales para generar nuevas. Por lo tanto, necesitamos diferentes tamaños de lotes para los modelos de entrenamiento previo y posterior.

Para esto, crearemos una función que nos permita reproducir modelos para diferentes tamaños de lote.

Hay tres capas en nuestro modelo:  

• Una capa Embedding: esta capa sirve como capa de entrada, acepta valores de entrada (en formato numerico) y los convierte en vectores.  
• Una capa GRU: una capa RNN llena con 1024 unidades de descenso de gradiente.  
• Una capa densa (Dense): para generar el resultado, con salidas vocab_size.

In [7]:
def construir_modelo(vocab_size, embedding_dim, rnn_units, batch_size):
    
    model = tf.keras.Sequential([
        
        tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim,
            batch_input_shape=[batch_size, None]),
        
        tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

#### Modelo para Entrenamiento

In [8]:
embedding_dim = 256
rnn_units = 1024

model = construir_modelo(
    vocab_size = len(vocab), # no. of uniques characters
    embedding_dim = embedding_dim, # 256
    rnn_units = rnn_units, # 1024
    batch_size = BATCH_SIZE) # 64 for the training

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           18688     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 73)            74825     
                                                                 
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


## Compilar y entrenar el modelo  

Optimizador: Adam
Funcion de perdida: sparse categorical crossentropy

Vectorizamos nuestro texto como números enteros (p. Ej., [0], [2], [1]), no en formato one-hot (p. Ej., [0,0,0], [0,1,], [1 , 0,0]). Para poder generar números enteros, debemos usar una función de entropía cruzada categórica dispersa (sparse categorical crossentropy).

In [9]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

Cargar los pesos y guardar el rendimiento de entrenamiento.

In [10]:
# Directory where the checkpoint will be saved
checkpoint_dir = './checkpoints'

# Name of the chekpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Entrenamiento del modelo

In [11]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
361/361 [==============================] - 58s 141ms/step - loss: 2.3067
Epoch 2/30
361/361 [==============================] - 52s 140ms/step - loss: 1.7582
Epoch 3/30
361/361 [==============================] - 52s 140ms/step - loss: 1.5594
Epoch 4/30
361/361 [==============================] - 52s 141ms/step - loss: 1.4490
Epoch 5/30
361/361 [==============================] - 52s 141ms/step - loss: 1.3703
Epoch 6/30
361/361 [==============================] - 52s 140ms/step - loss: 1.3032
Epoch 7/30
361/361 [==============================] - 52s 141ms/step - loss: 1.2436
Epoch 8/30
361/361 [==============================] - 52s 141ms/step - loss: 1.1880
Epoch 9/30
361/361 [==============================] - 52s 141ms/step - loss: 1.1378
Epoch 10/30
361/361 [==============================] - 52s 141ms/step - loss: 1.0916
Epoch 11/30
361/361 [==============================] - 52s 139ms/step - loss: 1.0519
Epoch 12/30
361/361 [==============================] - 52s 140ms/step - lo

## Generando texto con el modelo entrenado

In [12]:
# Ver la ubicación de nuestro último checkpoint
tf.train.latest_checkpoint(checkpoint_dir)

'./checkpoints/ckpt_30'

In [13]:
vocab_size = len(vocab)

# Ver la informacion del modelo
model = construir_modelo(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.save('gru_model_lyrics.h5') # Guardar el modelo completo
model.build(tf.TensorShape([1,None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            18688     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 73)             74825     
                                                                 
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


Funcion personalizada para preparar nuestra entrada para el modelo. Tenemos que configurar lo siguiente:  

• La cantidad de caracteres a generar.  
• Vectorizar la entrada (de cadena a numeros).  
• Una variable vacia para almacenar el resultado.  
• Un valor de temperatura para ajustar manualmente la variabilidad de las predicciones.  
• Desvectorizar la salida y tambien ingresar la salida al modelo nuevamente para la proxima prediccion.  
• Unir todos los caracteres generados para tener una cadena final.

In [14]:
def generar_texto(model, num_generate, temperature, start_string):
    input_eval = [char2idx[s] for s in start_string]   # de string a numero (vectorizacion)
    input_eval = tf.expand_dims(input_eval, 0)   # dimension
    texto_generado = []   # array vacio para guardar los resultados
    model.reset_states()   # limpiar los estados ocultos de la RNN

    for i in range(num_generate):
        predicciones = model(input_eval)   # prediccion para un solo caracter
        predicciones = tf.squeeze(predicciones, 0)  # remover el batch

        # usar una distribucion categorica para predecir el caracter devuelto por el modelo
        # una temperatura mas alta aumenta la probabilidad de seleccionar un caracter menos probable
        # mas bajo --> mas predecible
        predicciones = predicciones / temperature
        prediccion_id = tf.random.categorical(predicciones, num_samples=1)[-1,0].numpy()

        # El caracter predicho como la siguiente entrada al modelo
        # junto con el estado oculto anterior
        # Entonces el modelo hace la próxima prediccion basada en el caracter anterior
        input_eval = tf.expand_dims([prediccion_id], 0)
        # Desvectorizar el numero y agregar al texto generado
        texto_generado.append(idx2char[prediccion_id])

    return (start_string + ''.join(texto_generado))

Generar un texto

In [15]:
texto_generado = generar_texto(
                model,
                num_generate=500,
                temperature=1,
                start_string=u"fuego")

print(texto_generado)

fuego, que elegíder, te gana en blanca
saben que en esto yo lo mato a él
las perfectos
sobran ganas de rra me estrfeca
putas de freez, yeah

muerte parece mejor que me aman, he vida, más que amar
pero yo, lo quiero de ver el sol se están encerrados una cuerpo
y ya no tranco, los quieren moverte le exploque un paracias, ah


todo te la click de ese video', salgo con un pare' una de la pared
me va a gano hasta abajo buscando escándalo en el radio me han pegado que me digan qué pudieron
no puede quejar,


## Modelo usando los pesos guardados

In [16]:
# Nuevo modelo

embedding_dim = 256
rnn_units = 1024

model2 = construir_modelo(
    vocab_size = len(vocab), # no. of uniques characters
    embedding_dim = embedding_dim, # 256
    rnn_units = rnn_units, # 1024
    batch_size = BATCH_SIZE) # 64 for the training

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           18688     
                                                                 
 gru_2 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_2 (Dense)             (64, None, 73)            74825     
                                                                 
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [17]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model2.compile(optimizer='adam', loss=loss)

In [18]:
# Hacer predicciones con el nuevo modelo
from tensorflow.keras.models import load_model

model2 = load_model('gru_model_lyrics.h5')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            18688     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 73)             74825     
                                                                 
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Generar texto antes de cargar los pesos
texto_generado = generar_texto(
                model2,
                num_generate=500,
                temperature=1,
                start_string=u"fuego")

print(texto_generado)

fuego
estoy yendo pa' la cabeza

el barrio no puede ocupar mí llamada amaro, fue un encanto
me extraño porque dando suivan 
mirá como falta la sensei


me peo, éstropara en la escena virgen

ento a donde soy invicto, con un tembo de barrio
bardendo a todos mierda para que conoce

no se me pone quejos, tanto lugar suficiente
iba calle por donde aguanta cuando ataco lo pese'

yeh
yeh, yeh
yeh, yeh xanax, siempre ahora le hago confundo son roethow

to you keep limina
yeah code, code mí, dodel bebebaty
i
